In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.action_chains import ActionChains
import pyautogui as py
bus_details=[]

def busdetails(i,p):
   
    time.sleep(10)
    #c=driver.find_element(By.CSS_SELECTOR,'div[class="button"]')
    #action.move_to_element(c).click()
    wait=WebDriverWait(driver,100)
    try:
        child= driver.find_element(By.CSS_SELECTOR, "#result-section>div:nth-child(1) .button")
        driver.execute_script("arguments[0].scrollIntoView();", child)
        action.move_to_element(child).click().perform()
    except:
        print('no data')    
    
    try:
        element=driver.find_element(By.CSS_SELECTOR,"#result-section>div:nth-child(2) .button")
        driver.execute_script("arguments[0].scrollIntoView();", element)
        action.move_to_element(element).click().perform()
    except:
        print("nodatafound")    
    element = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.XPATH,'//div[@class="clearfix bus-item"]')))
    
    scrolling=True
    while element:
        old_page_source=driver.page_source
        body=driver.find_element(By.TAG_NAME,"body")
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(2)
        new_page_source=driver.page_source
        if new_page_source==old_page_source:
            element=False
    buses = driver.find_elements(By.XPATH,'//div[@class="clearfix bus-item-details"]')
    
    for bus in buses:
        name = bus.find_element(By.CSS_SELECTOR, ".travels").text
        type = bus.find_element(By.CSS_SELECTOR,".bus-type").text
        departure_time = bus.find_element(By.CSS_SELECTOR, ".dp-time").text
        arrival_time = bus.find_element(By.CSS_SELECTOR, ".bp-time").text
        duration = bus.find_element(By.CSS_SELECTOR, ".dur").text
        price = bus.find_element(By.CSS_SELECTOR, ".fare").text
        seats_available=bus.find_element(By.CSS_SELECTOR,".seat-left").text
        try:
            rating = bus.find_element(By.CSS_SELECTOR, ".rating-sec .rating").text
        except:
            rating = "No rating"

        bus_details.append({
            "route-collected": i,
            "name": name,
            "type":type,
            "departure_time": departure_time,
            "arrival_time": arrival_time,
            "duration": duration,
            "price": price,
            "seats_available":seats_available,
            "rating": rating
        })
        
    max_attempts = 10
    attempts = 0
    while attempts < max_attempts:
        py.hotkey('alt', 'left')
        time.sleep(1)
        try:
            WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CLASS_NAME, "route")))
            break
        except:
            attempts += 1
    
    scroll=driver.find_element(By.CSS_SELECTOR,f'.DC_117_paginationTable>.DC_117_pageTabs:nth-child({p})')        
    action.move_to_element(scroll).click().perform()      
       


driver=webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile")
action=ActionChains(driver)


p=1

for j in range(1,6):
    
    scroll=driver.find_element(By.CSS_SELECTOR,f'.DC_117_paginationTable>.DC_117_pageTabs:nth-child({j})')
    action.move_to_element(scroll).click().perform()
    route_collected=[]
    c=driver.find_elements(By.CLASS_NAME,"route")
    route_collected.extend([i.text for i in c])
    
    for i in route_collected:
        d=driver.find_element(By.LINK_TEXT,i)
        action.move_to_element(d).click().perform()
        busdetails(i,p)
   
    p+=1    
    


In [67]:
import pandas as pd
df=pd.DataFrame(bus_details)
df.to_csv("bihar.csv")
df.to_csv("assam.csv")
df.to_csv("telungana.csv")
df.to_csv("kerala.csv")
df.to_csv("aandhra.csv")
df.to_csv("kadamba.csv")
df.to_csv("delhi.csv")
df.to_csv("chandigar.csv")
df.to_csv("jammu.csv")
df.to_csv("rajasthan.csv")

In [68]:
import pandas as pd
aandra=pd.read_csv("aandra.csv")
assam=pd.read_csv('assam.csv')
bihar=pd.read_csv('bihar.csv')
chandigar=pd.read_csv('chandigar.csv')
delhi=pd.read_csv('Delhi.csv')
jammu=pd.read_csv('jammu.csv')
kadamba=pd.read_csv('kadmba.csv')
kerala=pd.read_csv('kerala.csv')
rajasthan=pd.read_csv('rajasthan.csv')
telungana=pd.read_csv('telungana.csv')
final=pd.concat([aandra,assam,bihar,chandigar,delhi,jammu,kadamba,kerala,rajasthan,telungana])

In [70]:
columns_to_convert = ['route-collected','name', 'type']
final[columns_to_convert]=final[columns_to_convert].astype('string')

In [71]:
def change_name(x):
    return x.str.replace(r'[A-Za-z]+', '', regex=True)

In [72]:
final['seats_available']=final['seats_available'].transform(change_name)
final['price']=final['price'].transform(change_name)

In [73]:
final['rating'] = pd.to_numeric(final['rating'], errors='coerce')
final['rating']=final['rating'].fillna(0)
final['rating']=final['rating'].astype('float64')

In [74]:
final['seats_available'] = pd.to_numeric(final['seats_available'], errors='coerce')
final['seats_available']=final['seats_available'].fillna(0)
final['seats_available']=final['seats_available'].astype('int64')

In [75]:
final['price'] = pd.to_numeric(final['price'], errors='coerce')
final['price']=final['price'].fillna(0)
final['price']=final['price'].astype('int64')

In [76]:
red_bus=final.drop(columns='Unnamed: 0')

In [77]:
red_bus.rename_axis('s_no', inplace=True)

In [ ]:
red_bus.to_csv("red_bus.csv")